In [2]:
#libraries required
import pandas as pd
#import mysql.connector
import string
import json
import psycopg2
from jsonpath_ng.ext import parse

In [2]:
#Functions
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#library that contains punctuation
#defining the function to remove punctuation
def remove_punctuation_and_lower(text):
    punctuationfree="".join([i.lower() for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text



def load_json_data(input_file):
    try:
        with open(input_file, 'r') as data_json:
            source_data = json.load(data_json)
        return source_data
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
        return None

In [3]:
# database = input("Enter database name:")
# host = input("Enter host name:")
# user = input("Enter user name:")
# password = input("Enter password:")
# port = input("Enter port number:")

conn = psycopg2.connect(database="nourish",
                        host="awesome-hw.sdsc.edu",
                        user="v1desai@ucsd.edu",
                        password="emfQGcx3",
                        port=5432)

cur = conn.cursor()

In [4]:
#product_ingredients_2018

list_of_tables = ['usda_2022_food_branded_experimental', 'usda_2022_branded_food_nutrients', 'usda_2022_nutrient_master']

cur.execute("""SELECT t1.fdc_id, t1.description, t2.nutrient_id, t2.amount, t3.name, t3.unit_name, t3.nutrient_nbr, t3.rank
            FROM usda_2022_food_branded_experimental 
            t1 LEFT JOIN usda_2022_branded_food_nutrients t2 ON t1.fdc_id = t2.fdc_id LEFT JOIN
            usda_2022_nutrient_master t3 ON t2.nutrient_id = t3.id LIMIT 100""")
join_records = cur.fetchall()
columns = [desc[0] for desc in cur.description]
join_df = pd.DataFrame(join_records, columns=columns)
group_object = join_df.groupby(['description'])['nutrient_nbr'].mean().reset_index()

In [5]:
pd.set_option('display.max_colwidth', None)
group_object.description



0    72% COCOA CRANBERRIES ALMONDS + DARK CHOCOLATE, CRANBERRIES ALMONDS
1                                           BEET WHOLE GRAIN CHIPS, BEET
2                                         MINT CHIP ICE CREAM, MINT CHIP
3                                              MUENSTER CHEESE, MUENSTER
4                       RASPBERRY CREAM TOP WHOLE MILK YOGURT, RASPBERRY
5                     STRAWBERRY ORGANIC TWISTED FRUIT SNACK, STRAWBERRY
6                                                  WHOLE PEELED TOMATOES
Name: description, dtype: object

In [6]:
for tbl in list_of_tables:
    table_type_column_query = f"""SELECT column_name, data_type 
                             FROM information_schema.columns
                             WHERE table_name = '{tbl}';"""
    cur.execute(table_type_column_query)
    records = cur.fetchall()
    print('\n')
    print(tbl)
    print(pd.DataFrame(records))




usda_2022_food_branded_experimental
                  0        1
0            fdc_id  integer
1         data_type     text
2       description     text
3  food_category_id     text
4  publication_date     text


usda_2022_branded_food_nutrients
                    0        1
0                  id  integer
1              fdc_id  integer
2         nutrient_id  integer
3              amount  numeric
4         data_points     text
5       derivation_id  integer
6                 min     text
7                 max     text
8              median     text
9            footnote     text
10  min_year_acquired     text


usda_2022_nutrient_master
              0        1
0            id  integer
1          name     text
2     unit_name     text
3  nutrient_nbr  numeric
4          rank     text


In [3]:
#prof deleted this table -- probably not good
#foods_to_avoid = pd.read_csv('Disease_foods_to_avoid_csv.csv', delimiter = ';')
#load in tasty ingredients and instructions as Pandas dataframe
with open('ingredient_and_instructions.json') as file:
    tasty_json_ingredients = json.load(file)
ingredients_and_recipes_df = pd.DataFrame(tasty_json_ingredients)


In [90]:
#load in tasty dishes (csv)
df_tasty_dishes = pd.read_csv('dishes.csv',usecols=['slug', 'protein', 'fat', 'calories', 'sugar', 'carbohydrates', 'fiber']) 
df_tasty_dishes.head()

,slug,protein,fat,calories,sugar,carbohydrates,fiber
0,homemade-cinnamon-rolls,7.0,21.0,479.0,24.0,63.0,1.0
1,whipped-coffee,0.0,0.0,69.0,18.0,18.0,0.0
2,fluffy-perfect-pancakes,36.0,50.0,1102.0,12.0,123.0,3.0
3,tasty-101-cinnamon-rolls,8.0,25.0,562.0,28.0,74.0,1.0
4,healthy-banana-pancakes,7.0,4.0,184.0,9.0,30.0,4.0


In [4]:
#unstructured data
jsonpath_expr = parse("$..['instructions']")
matches = [match.value for match in jsonpath_expr.find(tasty_json_ingredients)]

text = []
for i in range(len(matches)):
    list_text = " ".join(line['display_text'].strip() for line in matches[i])
    text.append(list_text)

In [11]:
text[100]

'Preheat oven to 350˚F (180˚C). Fill a greased muffin tin with your favorite combination of fillings or use the ones we did! Whisk eggs in measuring cup and fill up each tin. Sprinkle with salt and pepper. Bake for about 30 minutes or until a toothpick comes out clean. Enjoy!'

In [53]:
#extract all products
expr = parse("$")
list_products = [match.value.keys() for match in expr.find(tasty_json_ingredients)]


In [54]:
len(list(list_products[0]))

4961

In [5]:
jsonpath_expr = parse("$..['ingredient_sections']")
matches = [match.value for match in jsonpath_expr.find(tasty_json_ingredients)]

In [6]:
#extracting list of ingredients product by product
ingredients = []
ingredient_per_prod = []
for products in matches:
    for product in products:
        ing_per_prod = product['ingredients']
        for ing in ing_per_prod:
            ingredient_per_prod.append(ing['name'])
    ingredients.append(ingredient_per_prod)
    ingredient_per_prod = []

In [7]:
# extract all metric_unit for each ingredient per product
metric = []
metric_per_prod = []
for products in matches:
    for product in products:
        met_per_prod = product['ingredients']
        for met in met_per_prod:
            if met['metric_unit'] is not None:
                metric_per_prod.append((met['metric_unit']['quantity'],met['metric_unit']['display']))
            else:
                metric_per_prod.append((None,None))
    metric.append(metric_per_prod)
    metric_per_prod = []

In [62]:
pd.DataFrame(pd.Series(metric))

,0
0,"[(115, g), (480, mL), (100, g), (None, None), ..."
1,"[(28, g), (24, g), (12, g), (None, None), (Non..."
2,"[(500, g), (None, None), (960, mL), (170, g), ..."
3,"[(480, mL), (100, g), (None, None), (None, Non..."
4,"[(None, None), (None, None), (None, None), (70..."
...,...
4956,"[(185, g), (100, g), (None, None), (None, None..."
4957,"[(None, None), (None, None), (None, None), (No..."
4958,"[(None, None), (25, g), (60, g), (None, None),..."
4959,"[(None, None), (None, None), (None, None), (No..."


In [85]:
col_products = pd.Series(list_products[0])
col_ingredients = pd.Series(ingredients)
col_metric_unit = pd.Series(metric)
col_instructions = pd.Series(text)

In [86]:
pd.DataFrame(col_products)

,0
0,homemade-cinnamon-rolls
1,whipped-coffee
2,fluffy-perfect-pancakes
3,tasty-101-cinnamon-rolls
4,healthy-banana-pancakes
...,...
4956,instant-spinach-dhokla
4957,loaded-hot-dogs
4958,gluten-free-vegan-black-bean-burgers
4959,veggie-roast-and-tuna-salad


In [87]:
df = pd.DataFrame()
df['products'] = col_products
df['ingredients'] = col_ingredients
df['metric_unit'] = col_metric_unit
df['instructions'] = col_instructions
df.head()

,products,ingredients,metric_unit,instructions
0,homemade-cinnamon-rolls,"[unsalted butter, whole milk, granulated sugar...","[(115, g), (480, mL), (100, g), (None, None), ...",Generously butter two disposable foil pie/cake...
1,whipped-coffee,"[hot water, sugar, instant coffee powder, milk...","[(28, g), (24, g), (12, g), (None, None), (Non...","Add the hot water, sugar, and instant coffee t..."
2,fluffy-perfect-pancakes,"[flour, baking powder, milk, butter, egg yolks...","[(500, g), (None, None), (960, mL), (170, g), ...",Whisk together the flour and baking powder in ...
3,tasty-101-cinnamon-rolls,"[whole milk, sugar, unsalted butter, active dr...","[(480, mL), (100, g), (None, None), (None, Non...","Make the dough: In a large bowl, whisk togethe..."
4,healthy-banana-pancakes,"[ripe bananas, eggs, vanilla extract, quick-co...","[(None, None), (None, None), (None, None), (70...",Mash bananas in a large bowl until smooth. Mix...


In [97]:
df_products = df.merge(df_tasty_dishes,how='inner',left_on='products',right_on='slug')
df_products.drop('slug',axis=1,inplace=True)
df_products.head()

,products,ingredients,metric_unit,instructions,protein,fat,calories,sugar,carbohydrates,fiber
0,homemade-cinnamon-rolls,"[unsalted butter, whole milk, granulated sugar...","[(115, g), (480, mL), (100, g), (None, None), ...",Generously butter two disposable foil pie/cake...,7.0,21.0,479.0,24.0,63.0,1.0
1,whipped-coffee,"[hot water, sugar, instant coffee powder, milk...","[(28, g), (24, g), (12, g), (None, None), (Non...","Add the hot water, sugar, and instant coffee t...",0.0,0.0,69.0,18.0,18.0,0.0
2,fluffy-perfect-pancakes,"[flour, baking powder, milk, butter, egg yolks...","[(500, g), (None, None), (960, mL), (170, g), ...",Whisk together the flour and baking powder in ...,36.0,50.0,1102.0,12.0,123.0,3.0
3,tasty-101-cinnamon-rolls,"[whole milk, sugar, unsalted butter, active dr...","[(480, mL), (100, g), (None, None), (None, Non...","Make the dough: In a large bowl, whisk togethe...",8.0,25.0,562.0,28.0,74.0,1.0
4,healthy-banana-pancakes,"[ripe bananas, eggs, vanilla extract, quick-co...","[(None, None), (None, None), (None, None), (70...",Mash bananas in a large bowl until smooth. Mix...,7.0,4.0,184.0,9.0,30.0,4.0


In [72]:
tasty_dishes_filtered = tasty_dishes[['keywords', 'name', 'protein', 'fat', 'calories', 'sugar', 'carbohydrates', 'fiber']]